In [42]:
import ast
import json
import os
import sys
import timeit
import json
from collections import Counter


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from pathlib import Path
from brtdevkit.core.db.athena import AthenaClient
from brtdevkit.data import Dataset
import yaml

In [43]:
basepath = Path('/data/jupiter/datasets/')

In [44]:
dsetnames = [
    "Jupiter_20231019_HHH6_1615_1700",
    "Jupiter_20231121_HHH2_1800_1830",
    "Jupiter_20231026_HHH8_1515_1545",
    "Jupiter_20231007_HHH1_2350_0020",
    "Jupiter_20230926_HHH1_1815_1845",
    "Jupiter_20230927_HHH1_0100_0130",
    "Jupiter_20230814_HHH1_1415_1445",
    "Jupiter_20230825_HHH1_1730_1800",
    "Jupiter_20230803_HHH2_2030_2100",
    "Jupiter_20230803_HHH2_1400_1430",
    "Jupiter_20230803_HHH3_2115_2145",
    "Jupiter_20230720_HHH3_1805_1835",
    "Jupiter_20230823_HHH3_1815_1845",
]

In [45]:
os.listdir(basepath / dsetnames[0])

['online_calibration_data',
 'processed',
 'images',
 'annotations.csv',
 'master_annotations.csv',
 'partitions',
 'clean_ids.json']

In [46]:
for dsetname in dsetnames:
    print(dsetname)
    df = pd.read_csv(basepath / dsetname / 'master_annotations.csv')
    with open(basepath / dsetname / 'clean_ids.json', 'r') as f:
        uid = json.load(f)
    df[df['unique_id'].isin(uid)].drop_duplicates().to_csv(basepath / dsetname / 'master_annotations_clean.csv')

Jupiter_20231019_HHH6_1615_1700
Jupiter_20231121_HHH2_1800_1830
Jupiter_20231026_HHH8_1515_1545
Jupiter_20231007_HHH1_2350_0020
Jupiter_20230926_HHH1_1815_1845
Jupiter_20230927_HHH1_0100_0130
Jupiter_20230814_HHH1_1415_1445
Jupiter_20230825_HHH1_1730_1800
Jupiter_20230803_HHH2_2030_2100
Jupiter_20230803_HHH2_1400_1430
Jupiter_20230803_HHH3_2115_2145
Jupiter_20230720_HHH3_1805_1835
Jupiter_20230823_HHH3_1815_1845


In [ ]:
dsets = {}
for dsetname in dsetnames:
    print(dsetname)
    path = basepath / dsetname
    if not os.path.exists(path / 'annotations.csv'):
        print(f"Downloading to {path}")
        dset: Dataset = Dataset.retrieve(name=dsetname)
        df = dset.to_dataframe()
        dset.download(path)
        df.to_csv(path / 'annotations.csv')
    else:
        df = pd.read_csv(path / 'annotations.csv')
    dsets[dsetname] = {
        'df': df,
        'dpath': path,
    }


In [ ]:
from dl.dataset.pack_perception.download_ocal_data import download_ocal_data
download_ocal_data(str(path), df)

In [ ]:
from dl.dataset.pack_perception.ml_pack_perception import MLPackPerception
from dl.dataset.pack_perception.pack_perception_parameters import (
    PackPerceptionParameters,
)
dataset_path = basepath / dsetnames[0]
params = PackPerceptionParameters()
args = params.parser.parse_args(args=[
    '--csv-path', str(dataset_path / 'annotations.csv'),
    '--data-dir', str(dataset_path),
    '--cam-calibration-path', str(dataset_path / 'online_calibration_data'),
    '--batch-size', '4',
    '--multiprocess-workers', '32',
    '--pandarallel-workers', '32',
    '--gpu', '0',
    '--models', '512,768=ml_512x768_v3_full_rgb_halo_depth_10062023.ckpt', '512,640=ml_512x640_v3_full_rgb_halo_depth_10062023.ckpt'
    '--model-type' 'full' '--max-disp' '384' \
    '--run-oc''--image-only'
])

# MLPackPerception(vars(args))

In [ ]:
MLPackPerception(**vars(args))

In [ ]:
# Done already by stewart: 
# - Headlands removed
# - 

# Remove images with FPs

# Trim overlong sequences

# Choose sequences to meet criteria?